In [27]:
import polars as pl
from src.facilities_endpoint import fetch_facilities_data

df = await fetch_facilities_data()
if df is None:
    raise ValueError("df is None")
lf = df.lazy()
lf.collect()

facility_name,primo_id,primo_prprty,completion_name,division_name,division_id
str,i64,i64,str,str,i64
null,null,480001,"""AGGIELAND W""","""COLLEGE STATION""",19
null,null,110009,"""DTX W""","""DALLAS""",26
"""BAYOU CITY FAC""",69419,69420,"""BAYOU CITY 1""","""HOUSTON""",5
"""BAYOU CITY FAC""",69419,69421,"""BAYOU CITY 2""","""HOUSTON""",5
"""BAYOU CITY FAC""",69419,69422,"""BAYOU CITY 3""","""HOUSTON""",5
…,…,…,…,…,…
"""CAPITAL CITY FAC""",98750,98752,"""CAPITAL CITY 2""","""AUSTIN""",12
"""CAPITAL CITY FAC""",98750,98753,"""CAPITAL CITY 3""","""AUSTIN""",12
"""CAPITAL CITY FAC""",98750,98754,"""CAPITAL CITY 4""","""AUSTIN""",12


In [28]:
lf = lf.with_columns(pl.when(pl.col("primo_id").is_null()).then(pl.lit("WEL")).otherwise(pl.lit("FAC")).alias("entity_type"))
lf.collect()

facility_name,primo_id,primo_prprty,completion_name,division_name,division_id,entity_type
str,i64,i64,str,str,i64,str
null,null,480001,"""AGGIELAND W""","""COLLEGE STATION""",19,"""WEL"""
null,null,110009,"""DTX W""","""DALLAS""",26,"""WEL"""
"""BAYOU CITY FAC""",69419,69420,"""BAYOU CITY 1""","""HOUSTON""",5,"""FAC"""
"""BAYOU CITY FAC""",69419,69421,"""BAYOU CITY 2""","""HOUSTON""",5,"""FAC"""
"""BAYOU CITY FAC""",69419,69422,"""BAYOU CITY 3""","""HOUSTON""",5,"""FAC"""
…,…,…,…,…,…,…
"""CAPITAL CITY FAC""",98750,98752,"""CAPITAL CITY 2""","""AUSTIN""",12,"""FAC"""
"""CAPITAL CITY FAC""",98750,98753,"""CAPITAL CITY 3""","""AUSTIN""",12,"""FAC"""
"""CAPITAL CITY FAC""",98750,98754,"""CAPITAL CITY 4""","""AUSTIN""",12,"""FAC"""


In [29]:
lf = lf.with_columns(
    pl.coalesce(pl.col("facility_name"), pl.col("completion_name")).alias("entity_name"),
    pl.coalesce(pl.col("primo_id"), pl.col("primo_prprty")).alias("primo_id"),
)

lf = lf.drop("facility_name", "completion_name", "primo_prprty")
lf.collect()

primo_id,division_name,division_id,entity_type,entity_name
i64,str,i64,str,str
480001,"""COLLEGE STATION""",19,"""WEL""","""AGGIELAND W"""
110009,"""DALLAS""",26,"""WEL""","""DTX W"""
69419,"""HOUSTON""",5,"""FAC""","""BAYOU CITY FAC"""
69419,"""HOUSTON""",5,"""FAC""","""BAYOU CITY FAC"""
69419,"""HOUSTON""",5,"""FAC""","""BAYOU CITY FAC"""
…,…,…,…,…
98750,"""AUSTIN""",12,"""FAC""","""CAPITAL CITY FAC"""
98750,"""AUSTIN""",12,"""FAC""","""CAPITAL CITY FAC"""
98750,"""AUSTIN""",12,"""FAC""","""CAPITAL CITY FAC"""


In [30]:
lf = lf.unique(subset=["primo_id"])
lf = lf.sort(pl.col("division_id"))
lf.collect()

primo_id,division_name,division_id,entity_type,entity_name
i64,str,i64,str,str
69419,"""HOUSTON""",5,"""FAC""","""BAYOU CITY FAC"""
98750,"""AUSTIN""",12,"""FAC""","""CAPITAL CITY FAC"""
98743,"""AUSTIN""",12,"""FAC""","""ATX FAC"""
480001,"""COLLEGE STATION""",19,"""WEL""","""AGGIELAND W"""
110009,"""DALLAS""",26,"""WEL""","""DTX W"""
